In [2]:
import pandas as pd
import numpy as np

## DNN

In [2]:
data_dnn = pd.read_json('results_dnn.json').T

data_dnn

,n_epochs,batch_size,learning_rate,n_layers,activation_function,loss_function,optimizer,results
0,50,32,0.001,3,relu,Cross Entropy,ADAM,"[{'loss_values': [232.88340389728546, 219.4525..."
1,50,32,0.001,3,relu,Cross Entropy,SGD,"[{'loss_values': [239.34742295742035, 239.1654..."
2,50,32,0.001,3,relu,Cross Entropy,RMSprop,"[{'loss_values': [1311.3456435203552, 228.9097..."
3,50,32,0.001,3,relu,Multi Margin,ADAM,"[{'loss_values': [90.5774205327034, 81.7014083..."
4,50,32,0.001,3,relu,Multi Margin,SGD,"[{'loss_values': [105.32916295528412, 105.0802..."
...,...,...,...,...,...,...,...,...
139,50,64,0.1,15,sigmoid,Cross Entropy,SGD,"[{'loss_values': [164.53463542461395, 124.4378..."
140,50,64,0.1,15,sigmoid,Cross Entropy,RMSprop,"[{'loss_values': [12514.1414488554, 5595.03428..."
141,50,64,0.1,15,sigmoid,Multi Margin,ADAM,"[{'loss_values': [504.3324379324913, 226.14446..."
142,50,64,0.1,15,sigmoid,Multi Margin,SGD,"[{'loss_values': [67.28866493701935, 59.539265..."


In [3]:
metrics_dnn = []

for i, row in data_dnn.iterrows():

    accuracy_fold_train = []
    accuracy_fold_test = []

    recall_fold_train = []
    recall_fold_test = []

    precision_fold_train = []
    precision_fold_test = []

    for iteration in row["results"]:
        
        loss = iteration["loss_values"]
        conf_mat_train = np.array(iteration["confusion_matrix_train"])
        conf_mat_test = np.array(iteration["confusion_matrix_val"])
        
        # Compute accuracy (7 classes)
        total_train = conf_mat_train.sum()
        total_test = conf_mat_test.sum()

        accuracy_train = conf_mat_train.diagonal().sum() / total_train if total_train > 0 else 0
        accuracy_test = conf_mat_test.diagonal().sum() / total_test if total_test > 0 else 0

        # Compute recall and precision (per class)
        recall_train = np.array([
            conf_mat_train[i, i] / conf_mat_train[i, :].sum() if conf_mat_train[i, :].sum() > 0 else 0
            for i in range(7)
        ])
        recall_test = np.array([
            conf_mat_test[i, i] / conf_mat_test[i, :].sum() if conf_mat_test[i, :].sum() > 0 else 0
            for i in range(7)
        ])

        precision_train = np.array([
            conf_mat_train[i, i] / conf_mat_train[:, i].sum() if conf_mat_train[:, i].sum() > 0 else 0
            for i in range(7)
        ])
        precision_test = np.array([
            conf_mat_test[i, i] / conf_mat_test[:, i].sum() if conf_mat_test[:, i].sum() > 0 else 0
            for i in range(7)
        ])

        accuracy_fold_train.append(accuracy_train)
        accuracy_fold_test.append(accuracy_test)

        recall_fold_train.append(recall_train)
        recall_fold_test.append(recall_test)

        precision_fold_train.append(precision_train)
        precision_fold_test.append(precision_test)

    # Compute mean across folds
    accuracy_train = np.mean(accuracy_fold_train)
    accuracy_test = np.mean(accuracy_fold_test)

    recall_train = np.mean(recall_fold_train, axis=0)
    recall_test = np.mean(recall_fold_test, axis=0)

    precision_train = np.mean(precision_fold_train, axis=0)
    precision_test = np.mean(precision_fold_test, axis=0)

    average_recall_train = np.mean(recall_train)
    average_recall_test = np.mean(recall_test)

    average_precision_train = np.mean(precision_train)
    average_precision_test = np.mean(precision_test)

    f1_train_average = 2 * (average_precision_train * average_recall_train) / (average_precision_train + average_recall_train)
    f1_test_average = 2 * (average_precision_test * average_recall_test) / (average_precision_test + average_recall_test) 

    metrics_dnn.append({
        "iteration": i,
        "accuracy_train": accuracy_train,
        "accuracy_test": accuracy_test,
        "recall_train": recall_train.tolist(),  # Convert to list for DataFrame compatibility
        "recall_test": recall_test.tolist(),
        "averaged_recall_train": average_recall_train,
        "averaged_recall_test": average_recall_test,
        "precision_train": precision_train.tolist(),
        "precision_test": precision_test.tolist(),
        "averaged_precision_train": average_precision_train,
        "averaged_precision_test": average_precision_test,
        "f1_train_average": f1_train_average,
        "f1_test_average": f1_test_average
    })

metrics_dnn = pd.DataFrame(metrics_dnn)
metrics_dnn


,iteration,accuracy_train,accuracy_test,recall_train,recall_test,averaged_recall_train,averaged_recall_test,precision_train,precision_test,averaged_precision_train,averaged_precision_test,f1_train_average,f1_test_average
0,0,0.492194,0.452449,"[0.45666576580645024, 0.2781380793631726, 0.41...","[0.39492310575368617, 0.26124302397151344, 0.3...",0.492041,0.453465,"[0.3923431594126644, 0.4582671777499705, 0.382...","[0.35207534729986034, 0.3796717171717171, 0.35...",0.508026,0.459296,0.499906,0.456362
1,1,0.289694,0.276531,"[0.10461486097624861, 0.535960678336236, 0.091...","[0.0926535974196895, 0.540545522099994, 0.0831...",0.288489,0.281088,"[0.2850857940170572, 0.2471789073476965, 0.233...","[0.4244749380598437, 0.24591828080953, 0.17545...",0.330505,0.328967,0.308071,0.303149
2,2,0.469490,0.425714,"[0.22926222423077203, 0.2317780149045233, 0.46...","[0.17854807761175776, 0.18497423247048367, 0.4...",0.469501,0.425392,"[0.4713139798855225, 0.4143381572073497, 0.313...","[0.35781766024912004, 0.3630360986918364, 0.29...",0.492767,0.439270,0.480853,0.432220
3,3,0.500663,0.470204,"[0.42116714317564297, 0.3971881422840021, 0.37...","[0.3596384879068333, 0.3887226116726817, 0.346...",0.501354,0.467065,"[0.40255447100242014, 0.397145191949951, 0.459...","[0.3486435172522411, 0.38564581583879814, 0.41...",0.520917,0.484875,0.510949,0.475803
4,4,0.273878,0.260612,"[0.12795219917516099, 0.6912996932283679, 0.17...","[0.12190061903965199, 0.6954819776464157, 0.16...",0.271799,0.268520,"[0.22030669708290626, 0.2214708161247449, 0.13...","[0.10364346128822381, 0.22680118760984924, 0.1...",0.297552,0.239952,0.284093,0.253433
...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,139,0.143418,0.140612,"[0.2, 0.0, 0.0, 0.0, 0.0, 0.8, 0.0]","[0.2, 0.0, 0.0, 0.0, 0.0, 0.8, 0.0]",0.142857,0.142857,"[0.028775510204081634, 0.0, 0.0, 0.0, 0.0, 0.1...","[0.027755102040816326, 0.0, 0.0, 0.0, 0.0, 0.1...",0.020488,0.020087,0.035837,0.035222
140,140,0.143265,0.141224,"[0.2, 0.2, 0.0, 0.2, 0.0, 0.4, 0.0]","[0.2, 0.2, 0.0, 0.2, 0.0, 0.4, 0.0]",0.142857,0.142857,"[0.028775510204081634, 0.02811224489795918, 0....","[0.027755102040816326, 0.03040816326530612, 0....",0.020466,0.020175,0.035804,0.035357
141,141,0.141837,0.146939,"[0.0, 0.2, 0.2, 0.0, 0.2, 0.2, 0.2]","[0.0, 0.2, 0.2, 0.0, 0.2, 0.2, 0.2]",0.142857,0.142857,"[0.0, 0.02857142857142857, 0.02806122448979591...","[0.0, 0.02857142857142857, 0.03061224489795918...",0.020262,0.020991,0.035491,0.036604
142,142,0.146122,0.129796,"[0.2, 0.2, 0.2, 0.0, 0.2, 0.2, 0.0]","[0.2, 0.2, 0.2, 0.0, 0.2, 0.2, 0.0]",0.142857,0.142857,"[0.028775510204081634, 0.028520408163265305, 0...","[0.027755102040816326, 0.028775510204081634, 0...",0.020875,0.018542,0.036427,0.032824


In [4]:
# get the max value for f1-score

max_f1_test = metrics_dnn["f1_test_average"].max()
max_f1_train = metrics_dnn["f1_train_average"].max()
max_f1_test, max_f1_train

(np.float64(0.49206776874351155), np.float64(0.5411958702807196))

## CNN

In [5]:
data_cnn = pd.read_json('results_cnn_1.json').T

data_cnn

,n_epochs,activation_function,pooling,n_conv_layers,conv_out_channels,conv_kernel_size,conv_padding,n_layers,batch_size,learning_rate,loss_function,optimizer,results
0,50,relu,MaxPool,1,16,3,1,3,32,0.001,Cross Entropy,ADAM,"[{'loss_values': [231.8701878786087, 197.12700..."
1,50,relu,MaxPool,1,16,3,1,3,32,0.001,Cross Entropy,SGD,"[{'loss_values': [239.3160616159439, 239.26000..."
2,50,relu,MaxPool,1,16,3,1,3,32,0.001,Cross Entropy,RMSprop,"[{'loss_values': [5110.887897610664, 220.26268..."
3,50,relu,MaxPool,1,16,3,1,3,32,0.001,Multi Margin,ADAM,"[{'loss_values': [86.5283233821392, 55.5496368..."
4,50,relu,MaxPool,1,16,3,1,3,32,0.001,Multi Margin,SGD,"[{'loss_values': [105.38043284416199, 105.3138..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
571,50,sigmoid,AvgPool,2,16,3,1,15,64,0.1,Cross Entropy,SGD,"[{'loss_values': [283.7384283542633, 244.37010..."
572,50,sigmoid,AvgPool,2,16,3,1,15,64,0.1,Cross Entropy,RMSprop,"[{'loss_values': [12106.481934070587, 6383.127..."
573,50,sigmoid,AvgPool,2,16,3,1,15,64,0.1,Multi Margin,ADAM,"[{'loss_values': [630.6281946897507, 447.19764..."
574,50,sigmoid,AvgPool,2,16,3,1,15,64,0.1,Multi Margin,SGD,"[{'loss_values': [123.1793560385704, 111.78310..."


In [19]:
def get_metrics(data : pd.DataFrame):

    metrics_cnn = []

    for i, row in data.iterrows():

        accuracy_fold_train = []
        accuracy_fold_test = []

        recall_fold_train = []
        recall_fold_test = []

        precision_fold_train = []
        precision_fold_test = []

        for iteration in row["results"]:
            
            loss = iteration["loss_values"]
            conf_mat_train = np.array(iteration["confusion_matrix_train"])
            conf_mat_test = np.array(iteration["confusion_matrix_val"])
            
            # Compute accuracy (7 classes)
            total_train = conf_mat_train.sum()
            total_test = conf_mat_test.sum()

            accuracy_train = conf_mat_train.diagonal().sum() / total_train if total_train > 0 else 0
            accuracy_test = conf_mat_test.diagonal().sum() / total_test if total_test > 0 else 0

            # Compute recall and precision (per class)
            recall_train = np.array([
                conf_mat_train[i, i] / conf_mat_train[i, :].sum() if conf_mat_train[i, :].sum() > 0 else 0
                for i in range(7)
            ])
            recall_test = np.array([
                conf_mat_test[i, i] / conf_mat_test[i, :].sum() if conf_mat_test[i, :].sum() > 0 else 0
                for i in range(7)
            ])

            precision_train = np.array([
                conf_mat_train[i, i] / conf_mat_train[:, i].sum() if conf_mat_train[:, i].sum() > 0 else 0
                for i in range(7)
            ])
            precision_test = np.array([
                conf_mat_test[i, i] / conf_mat_test[:, i].sum() if conf_mat_test[:, i].sum() > 0 else 0
                for i in range(7)
            ])

            accuracy_fold_train.append(accuracy_train)
            accuracy_fold_test.append(accuracy_test)

            recall_fold_train.append(recall_train)
            recall_fold_test.append(recall_test)

            precision_fold_train.append(precision_train)
            precision_fold_test.append(precision_test)

        # Compute mean across folds
        accuracy_train = np.mean(accuracy_fold_train)
        accuracy_test = np.mean(accuracy_fold_test)

        recall_train = np.mean(recall_fold_train, axis=0)
        recall_test = np.mean(recall_fold_test, axis=0)

        precision_train = np.mean(precision_fold_train, axis=0)
        precision_test = np.mean(precision_fold_test, axis=0)

        average_recall_train = np.mean(recall_train)
        average_recall_test = np.mean(recall_test)

        average_precision_train = np.mean(precision_train)
        average_precision_test = np.mean(precision_test)

        f1_train_average = 2 * (average_precision_train * average_recall_train) / (average_precision_train + average_recall_train)
        f1_test_average = 2 * (average_precision_test * average_recall_test) / (average_precision_test + average_recall_test) 

        row = row.drop("results")
        row_dict = row.to_dict()

        metrics_cnn.append({
            **row_dict,
            "iteration": i,
            "accuracy_train": accuracy_train,
            "accuracy_test": accuracy_test,
            "recall_train": recall_train.tolist(),  # Convert to list for DataFrame compatibility
            "recall_test": recall_test.tolist(),
            "averaged_recall_train": average_recall_train,
            "averaged_recall_test": average_recall_test,
            "precision_train": precision_train.tolist(),
            "precision_test": precision_test.tolist(),
            "averaged_precision_train": average_precision_train,
            "averaged_precision_test": average_precision_test,
            "f1_train_average": f1_train_average,
            "f1_test_average": f1_test_average
        })

    metrics_cnn = pd.DataFrame(metrics_cnn)
    return metrics_cnn


In [20]:
metrics_cnn = get_metrics(data_cnn)

In [21]:
max_f1_train = metrics_cnn["f1_train_average"].max()
max_f1_test = metrics_cnn["f1_test_average"].max()

max_f1_test, max_f1_train

(np.float64(0.6551872819161952), np.float64(0.973445018550672))

In [22]:
max_f1_test_df = metrics_cnn[metrics_cnn["f1_test_average"] == max_f1_test]

max_f1_test_df

,n_epochs,activation_function,pooling,n_conv_layers,conv_out_channels,conv_kernel_size,conv_padding,n_layers,batch_size,learning_rate,...,recall_train,recall_test,averaged_recall_train,averaged_recall_test,precision_train,precision_test,averaged_precision_train,averaged_precision_test,f1_train_average,f1_test_average
88,50,relu,MaxPool,2,16,3,1,3,32,0.1,...,"[0.8659264478857184, 0.8888974790428635, 0.887...","[0.5892674918450334, 0.5782651828005414, 0.601...",0.887827,0.651995,"[0.8860410350767609, 0.8516981988488196, 0.815...","[0.6093338004601769, 0.5575159327129008, 0.500...",0.89204,0.658411,0.889929,0.655187


In [23]:
# get 50 best performances in terms of test accuracy
best_cnn = metrics_cnn.sort_values(by="accuracy_test", ascending=False).head(50)
data_cnn_50_best = data_cnn.loc[best_cnn.index]
data_cnn_50_best.to_csv("data_cnn_50_best.csv")

In [18]:
loss_max_f1_test = data_cnn.iloc[88]["results"][0]["loss_values"]

NameError: name 'data_cnn' is not defined

## Early Stopping

In [16]:
data_cnn_early_stopping = pd.read_json('results_cnn_early.json').T
#data_cnn_early_stopping

In [26]:
metrics_early = get_metrics(data_cnn_early_stopping)

In [28]:
max_f1_train = metrics_early["f1_train_average"].max()
max_f1_test = metrics_early["f1_test_average"].max()

max_f1_test, max_f1_train

(np.float64(0.6309427591407378), np.float64(0.8425808460199662))

## No Data Augmentation

In [17]:
data_no_aug_cnn = pd.read_json('../results_original/results_cnn.json').T

#data_no_aug_cnn

In [22]:
import ast

results_all = []
j = 0

for i, row in data_no_aug_cnn.iterrows():

    conf_mat_train = np.array(row["confusion_matrix_train"])
    conf_mat_test = np.array(row["confusion_matrix_val"])

    total_train = conf_mat_train.sum()
    total_test = conf_mat_test.sum()

    accuracy_train = conf_mat_train.diagonal().sum() / total_train if total_train > 0 else 0
    accuracy_test = conf_mat_test.diagonal().sum() / total_test if total_test > 0 else 0

    # Compute recall and precision (per class)
    recall_train = np.array([
        conf_mat_train[i, i] / conf_mat_train[i, :].sum() if conf_mat_train[i, :].sum() > 0 else 0
        for i in range(7)
    ])
    recall_test = np.array([
        conf_mat_test[i, i] / conf_mat_test[i, :].sum() if conf_mat_test[i, :].sum() > 0 else 0
        for i in range(7)
    ])

    precision_train = np.array([
        conf_mat_train[i, i] / conf_mat_train[:, i].sum() if conf_mat_train[:, i].sum() > 0 else 0
        for i in range(7)
    ])
    precision_test = np.array([
        conf_mat_test[i, i] / conf_mat_test[:, i].sum() if conf_mat_test[:, i].sum() > 0 else 0
        for i in range(7)
    ])

    average_recall_train = np.mean(recall_train)
    average_precision_train = np.mean(precision_train)

    average_recall_test = np.mean(recall_test)
    average_precision_test = np.mean(precision_test)

    f1_score_train = 2 * (average_precision_train * average_recall_train) / (average_precision_train + average_recall_train)
    f1_score_test  = 2 * (average_precision_test  * average_recall_test)  / (average_precision_test  + average_recall_test)

    results_all.append({
        "iteration": j,
        "accuracy_train": accuracy_train,
        "accuracy_test": accuracy_test,
        "recall_train": recall_train.tolist(),
        "recall_test": recall_test.tolist(),
        "averaged_recall_train": average_precision_train,
        "averaged_recall_test": average_recall_test,
        "precision_train": precision_train,
        "precision_test": precision_test,
        "averaged_precision_train": average_precision_train,
        "averaged_precision_test": average_precision_test,
        "f1_score_train": f1_score_train,
        "f1_score_test": f1_score_test
    })
    
    j += 1

metrics_cnn_no_aug = pd.DataFrame(results_all)
#metrics_cnn_no_aug

## Only undersampling

In [7]:
data_under_cnn = pd.read_json('./results_cnn_under.json').T

In [21]:
metrics_under_cnn = get_metrics(data_under_cnn)

In [24]:
max_f1_train = metrics_under_cnn["f1_train_average"].max()
max_f1_test = metrics_under_cnn["f1_test_average"].max()

max_f1_test, max_f1_train

(np.float64(0.4991124052880161), np.float64(0.8951487107489473))